In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#!/usr/bin/env python
# coding: utf-8

import os,sys
sys.path.insert(0,"..")
from glob import glob
from os.path import exists, join
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import argparse

import torch
import torchvision, torchvision.transforms
import skimage.transform
import sklearn

import random
import train_utils
import pickle
import torchxrayvision as xrv


In [27]:
!ls res*

results_chex-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best_chex.pkl
results_kaggle-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best_kaggle.pkl
results_mimic_ch-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best_mimic_ch.pkl
results_mimic_nb-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best_mimic_nb.pkl
results_nih-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best_nih.pkl
results_nih-pc-chex-mimic_ch-google-openi-kaggle-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best_nih-pc-chex-mimic_ch-google-openi-kaggle.pkl
results_pc-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best_pc.pkl


In [26]:
dataset_name = "nih-pc-chex-mimic_ch-google-openi-kaggle"
results = pickle.load(open("results_{}-densenet121-d121-tw-lr001-rot45-tr15-sc15-seed0-best_{}.pkl".format(dataset_name, dataset_name), "br"))


In [45]:
all_threshs = []
all_min = []
all_max = []
all_ppv80 = []
for i, patho in enumerate(xrv.datasets.default_pathologies):
    opt_thres = np.nan
    opt_max = np.nan
    if len(results[3][i]) > 0:
        
        #sigmoid
        all_outputs = 1.0/(1.0 + np.exp(-results[2][i]))
        
        fpr, tpr, thres = sklearn.metrics.roc_curve(results[3][i], all_outputs)
        pente = tpr - fpr
        opt_thres = thres[np.argmax(pente)]
        opt_min = all_outputs.min()
        opt_max = all_outputs.max()
        
        ppv, recall, thres = sklearn.metrics.precision_recall_curve(results[3][i], all_outputs)
        ppv80_thres_idx = np.where(ppv > 0.8)[0][0]
        ppv80_thres = thres[ppv80_thres_idx-1]
        
        
    all_threshs.append(opt_thres)
    all_min.append(opt_min)
    all_max.append(opt_max)
    all_ppv80.append(ppv80_thres)

In [46]:
print(str(all_threshs).replace("nan","np.nan"))

[0.07422872, 0.038290843, 0.09814756, 0.0098118475, 0.023601074, 0.0022490358, 0.010060724, 0.103246614, 0.056810737, 0.026791653, 0.050318155, 0.023985857, 0.01939503, 0.042889766, 0.053369623, 0.035975814, 0.20204692, 0.05015312]


In [47]:
print(str(all_min).replace("nan","np.nan"))

[2.9620523e-05, 1.8223789e-05, 0.00051482924, 1.8099824e-07, 5.410019e-09, 8.306624e-09, 1.9041477e-09, 1.1754888e-05, 9.194446e-07, 3.6992893e-07, 5.6234506e-07, 5.929607e-05, 1.1718594e-06, 1.2792777e-08, 5.942449e-07, 1.1029978e-05, 0.0006902651, 9.664481e-05]


In [48]:
print(str(all_max).replace("nan","np.nan"))

[0.9676907, 0.9329898, 0.9454921, 0.691005, 0.95700103, 0.46127197, 0.8549093, 0.99547607, 0.9962031, 0.6920735, 0.9889571, 0.8832056, 0.9939653, 0.9999714, 0.9900496, 0.90185666, 0.9943041, 0.9523041]


In [49]:
print(str(all_ppv80).replace("nan","np.nan"))

[0.72715247, 0.8885005, 0.92493945, 0.6527224, 0.68707734, 0.46127197, 0.7272054, 0.6127343, 0.9878492, 0.61979693, 0.66309816, 0.7853459, 0.930661, 0.93645346, 0.6788558, 0.6547198, 0.61614525, 0.8489876]


In [41]:
ppv, recall, thres = sklearn.metrics.precision_recall_curve(results[3][0], results[2][0], pos_label=1)

In [36]:
ppv80_thres = np.where(ppv > 0.8)[0][0]

In [37]:
npv80_thres = np.where(npv > 0.8)[0][0]

In [42]:
thres[ppv80_thres]

0.98024523